In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In this proejcts, I would like to combine different data tables using the SQL "join" feature and analyze which country has spent the most and least money on education over a certain time period.

Let's first import Google's bigquery module to use this World bank dataset on my Kaggle kernel.

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

Loading the dataset..

In [ ]:
dataset_ref = client.dataset('world_bank_intl_education', project='bigquery-public-data')

In [ ]:
wb_dset = client.get_dataset(dataset_ref)

In [ ]:
type(wb_dset)

Let's take a look at the list of tables we have in this "wb_dset" dataset.

In [ ]:
[x.table_id for x in client.list_tables(wb_dset)]

We have a total of 4 tables. Let's take a look at the schema for "country_series_definitions" table.

In [ ]:
wb_full = client.get_table(wb_dset.table('country_series_definitions'))

In [ ]:
type(wb_full)

In [ ]:
wb_full.schema

Working fine! Let's take a look at the top 5 rows of each table.

In [ ]:
query = """SELECT *
                FROM `bigquery-public-data.world_bank_intl_education.country_series_definitions`
                LIMIT 5
                """

query_job = client.query(query)
query_job.to_dataframe()

In [ ]:
query = """SELECT *
                FROM `bigquery-public-data.world_bank_intl_education.country_summary`
                LIMIT 5
                """

query_job = client.query(query)
query_job.to_dataframe()

In [ ]:
query = """SELECT *
                FROM `bigquery-public-data.world_bank_intl_education.international_education`
                LIMIT 5
                """

query_job = client.query(query)
query_job.to_dataframe()

Before we proceed further, I would like to check if the "series_code" key in the "country_series_definitions" table is the same as the "indicator_code" in the "internation_education" table.

For this, let's see if there are any rows in the country_series_definitions table that match with the indicator_code in the international_education table. (without using JOIN)

In [ ]:
query = """SELECT A.country_code, A.series_code
                FROM `bigquery-public-data.world_bank_intl_education.country_series_definitions` A
                WHERE A.series_code in (select B.indicator_code from `bigquery-public-data.world_bank_intl_education.international_education` B)
                ORDER by A.country_code
                LIMIT 20
                """

query_job = client.query(query)
query_job.to_dataframe()

We can see some records match between two tables. Let's confirm this one more time with Inner Join.

In [ ]:
query = """SELECT A.country_code, A.description, A.series_code, B.indicator_code
                FROM `bigquery-public-data.world_bank_intl_education.international_education` B
                JOIN `bigquery-public-data.world_bank_intl_education.country_series_definitions` A ON (B.indicator_code=A.series_code)
                ORDER by A.country_code
                LIMIT 100
                """

query_job = client.query(query)
query_job.to_dataframe()

"JOIN" also resulted some records that share the same code between two tables. It seems we are getting many duplicates for each country, probably due to different data sources. For simplicity, let's just take a look at distinct sets.

In [ ]:
query = """SELECT DISTINCT A.country_code, A.description, A.series_code, B.indicator_code
                FROM `bigquery-public-data.world_bank_intl_education.international_education` B
                JOIN `bigquery-public-data.world_bank_intl_education.country_series_definitions` A ON (B.indicator_code=A.series_code)
                ORDER by A.country_code
                LIMIT 100
                """

query_job = client.query(query)
query_job.to_dataframe()

Much clearer now.

Let's join country_series_definitions, country_code and international_education tables on the country_code key to take a deeper look at educatication budget data.

In [ ]:
query = """SELECT DISTINCT A.country_code, B.short_name, A.series_code, A.description, C.value, C.year
                FROM `bigquery-public-data.world_bank_intl_education.country_series_definitions` A
                JOIN `bigquery-public-data.world_bank_intl_education.country_summary` B ON A.country_code = B.country_code
                JOIN `bigquery-public-data.world_bank_intl_education.international_education` C ON A.country_code = C.country_code
                ORDER by A.country_code, C.year DESC
                LIMIT 10
                """

query_job = client.query(query)
query_job.to_dataframe()

Hmm. I did not notice this until now but it seems we have some "projection" data for future. Let's ignore them for now for the sake of historical data analysis.

Before doing so let's look at the distribution of data in terms of year.

In [ ]:
query = """SELECT DISTINCT C.year, count(*)
                FROM `bigquery-public-data.world_bank_intl_education.country_series_definitions` A
                JOIN `bigquery-public-data.world_bank_intl_education.country_summary` B ON A.country_code = B.country_code
                JOIN `bigquery-public-data.world_bank_intl_education.international_education` C ON A.country_code = C.country_code
                GROUP BY C.year
                ORDER by C.year
                """

query_job = client.query(query)
query_job.to_dataframe()

We can see that it goes up to year '2100'. Let's cut it at '2019'.

In [ ]:
query = """SELECT DISTINCT A.country_code, B.short_name, A.series_code, A.description, C.value, C.year
                FROM `bigquery-public-data.world_bank_intl_education.country_series_definitions` A
                JOIN `bigquery-public-data.world_bank_intl_education.country_summary` B ON A.country_code = B.country_code
                JOIN `bigquery-public-data.world_bank_intl_education.international_education` C ON A.country_code = C.country_code
                WHERE C.year < 2019
                ORDER by C.year DESC
                LIMIT 10
                """

query_job = client.query(query)
query_job.to_dataframe()

Now we can see that "year" does not go above 2017. How far does this table go back to?

In [ ]:
query = """SELECT min(C.year)
                FROM `bigquery-public-data.world_bank_intl_education.country_series_definitions` A
                JOIN `bigquery-public-data.world_bank_intl_education.country_summary` B ON A.country_code = B.country_code
                JOIN `bigquery-public-data.world_bank_intl_education.international_education` C ON A.country_code = C.country_code
                """

query_job = client.query(query)
query_job.to_dataframe()

Ok so now our dataset starts from 1970 and goes until 2017.

Let's look at how much money each country has spent during this period of 1970-2017.

In [ ]:
query = """SELECT DISTINCT A.country_code, B.short_name, sum(C.value) as Education_sum
                FROM `bigquery-public-data.world_bank_intl_education.country_series_definitions` A
                JOIN `bigquery-public-data.world_bank_intl_education.country_summary` B ON A.country_code = B.country_code
                JOIN `bigquery-public-data.world_bank_intl_education.international_education` C ON A.country_code = C.country_code
                WHERE C.year < 2019
                GROUP BY A.country_code, B.short_name
                ORDER BY Education_sum DESC
                LIMIT 10
                """

query_job = client.query(query)
query_job.to_dataframe()

You can see China overall has spent the most on education during the 1970-2017 period.
Which country has spent the least on education during this period then?

In [ ]:
query = """SELECT DISTINCT A.country_code, B.short_name, sum(C.value) as Education_sum
                FROM `bigquery-public-data.world_bank_intl_education.country_series_definitions` A
                JOIN `bigquery-public-data.world_bank_intl_education.country_summary` B ON A.country_code = B.country_code
                JOIN `bigquery-public-data.world_bank_intl_education.international_education` C ON A.country_code = C.country_code
                WHERE C.year < 2019
                GROUP BY A.country_code, B.short_name
                ORDER BY Education_sum
                LIMIT 1
                """

query_job = client.query(query)
query_job.to_dataframe()

Honestly I have not heard of this country but this country has spent about 9 orders of mangitude less on education than China has done during the same period.

This dataset overall seems like an interesting project to work on given that it even has future data. 